### SQLite Mini Project

The goal of this project is to export the country club data from PHPMyAdmin, connect to a local SQLite instance from Jupyter notebook and answer several questions through SQL code. 

In [2]:
# Import sqlite
import sqlite3
import pandas as pd

# initiate read-write connection
conn = sqlite3.connect('sqlite_db_pythonsqlite.db', uri=True)

# Create cursor
curs = conn.cursor()

# Fetch all from master
curs.execute("SELECT name FROM sqlite_master WHERE type = 'table'").fetchall()

[('Bookings',), ('Facilities',), ('Members',)]

Every SQLite database has an SQLite Master table that defines the schema for the database. If all of the relevant tables from the country club database are in the master table, then the connection was successful. The fetchall method from the cursor found the same tables that were in the phpmyadmin app.

#### QUESTIONS:

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [3]:
# Fetch name, total_rev from facilities. Order by Rev, less than 1000.

# for row in curs.execute("PRAGMA table_info(Facilities)"):
 #   print(row)

# for row in curs.execute("PRAGMA table_info(Bookings)"):
#    print(row)
    
# Revenue is the money made without cost. Create query

query_revenue = "SELECT name, revenue" \
"FROM" \
" (  SELECT name," \
           "SUM("  \
          	"CASE WHEN b.memid = 0 THEN f.guestcost * b.slots" \
		   " ELSE f.membercost * b.slots END" \
           ") AS revenue" \
   " FROM Bookings AS b" \
    "JOIN Facilities AS f" \
	"ON B.facid = F.facid" \
    "GROUP BY name) subquery" \
"WHERE revenue < 1000" 

# Use cursor to execute 
result_revenue = curs.execute("SELECT name, revenue FROM(  SELECT name, SUM(CASE WHEN b.memid = 0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END ) AS revenue FROM Bookings AS b JOIN Facilities AS f ON B.facid = F.facid GROUP BY name) subquery WHERE revenue < 1000").fetchall()

# Convert to dataframe
df_revenue = pd.DataFrame(result_revenue, 
                          columns=["Name", "Revenue"])

df_revenue

,Name,Revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order.

In [4]:
# No CONCAT in sqlite, use ||

# Create query
query_recommended = "SELECT (i.surname ||' '|| i.firstname) AS Member," \
"(m.surname ||' '|| m.firstname) AS Recommender " \
"FROM Members AS m" \
"INNER JOIN Members AS i" \
"ON m.memid = i.recommendedby" \
"WHERE m.memid > 0;" \
"ORDER BY Member;"

# Execute through cursor
result_recommended = curs.execute("SELECT (i.surname ||', '|| i.firstname) AS Member, (m.surname ||', '|| m.firstname) AS Recommender FROM Members AS m INNER JOIN Members AS i ON m.memid = i.recommendedby WHERE m.memid > 0 ORDER BY Member;").fetchall()

# Convert to dataframe
df_recommended = pd.DataFrame(result_recommended, 
                             columns=["Member","Recommender"])

df_recommended

,Member,Recommender
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


Q12: Find the facilities with their usage by member, but not guests.

In [5]:
# In future, use PIVOT on subquery for better legiibility for facility

# Create query
query_usage = "SELECT" \
"m.firstname ||' '|| m.surname AS Member," \
"f.name AS Facility," \
"COUNT(b.facid) AS Utilization" \
"FROM Bookings AS b" \
"JOIN Facilities AS f" \
"ON b.facid = f.facid" \
"JOIN Members AS m" \
"ON b.memid = m.memid" \
"WHERE b.memid > 0" \
"GROUP BY b.memid, b.facid" \
"ORDER BY Facility;"

# Cursor to execute query
result_usage = curs.execute("SELECT m.firstname ||' '|| m.surname AS Member, f.name AS Facility, COUNT(b.facid) AS Utilization FROM Bookings AS b JOIN Facilities AS f ON b.facid = f.facid JOIN Members AS m ON b.memid = m.memid WHERE b.memid > 0 GROUP BY b.memid, b.facid ORDER BY Facility;").fetchall()

# Convert to dataframe
df_usage = pd.DataFrame(result_usage, 
                        columns=["Name", "Facility", "Usage"])

df_usage.head(15)

,Name,Facility,Usage
0,Darren Smith,Badminton Court,132
1,Tracy Smith,Badminton Court,32
2,Tim Rownam,Badminton Court,4
3,Gerald Butters,Badminton Court,20
4,Burton Tracy,Badminton Court,2
5,Nancy Dare,Badminton Court,10
6,Tim Boothe,Badminton Court,12
7,Ponder Stibbons,Badminton Court,16
8,Charles Owen,Badminton Court,6
9,David Jones,Badminton Court,8


Q13: Find the facilities usage by month, but not guests.

In [20]:
# Create query
query_monthlyusage = "SELECT f.name AS Name, " \
"SUM(b.slots) AS Utilization," \
"STRFTIME('%m', b.starttime) as Month " \
 "FROM Bookings AS b" \
 "JOIN Facilities AS f" \
 "ON b.facid = f.facid" \
 "WHERE b.memid <> 0" \
 "GROUP BY Month, b.facid" \
 "ORDER BY Month, b.facid" \

# Cursor to execute code
result_monthlyusage = curs.execute("SELECT f.name, SUM(b.slots), STRFTIME('%m', b.starttime) as Month FROM Bookings AS b JOIN Facilities AS f ON b.facid = f.facid WHERE b.memid <> 0 GROUP BY Month, b.facid ORDER BY Month, b.facid;").fetchall()

# Convert to dataframe
df_monthlyusage = pd.DataFrame(result_monthlyusage, 
                              columns = ["Name", "Usage", "Month"])

df_monthlyusage

,Name,Usage,Month
0,Tennis Court 1,201,07
1,Tennis Court 2,123,07
2,Badminton Court,165,07
3,Table Tennis,98,07
4,Massage Room 1,166,07
5,Massage Room 2,8,07
6,Squash Court,50,07
7,Snooker Table,140,07
8,Pool Table,110,07
9,Tennis Court 1,339,08
